In [ ]:
!pip -q install langchain openai pinecone langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
from google.colab import userdata
pinecone_api = userdata.get('PINECONE_API_KEY')

In [ ]:
from pinecone import ServerlessSpec

In [ ]:
spec = ServerlessSpec(
    cloud  = "aws",
    region = "us-east-1"
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# load embedding
embeddings  = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")
embed_model = embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
embed = embed_model.embed_documents("ฝนตกหนักถล่มภาพเหนือ")
len(embed),len(embed[0])

(20, 1024)

In [ ]:
import time
from pinecone import Pinecone

In [ ]:
pc = Pinecone(api_key = pinecone_api)
index_name = "us-place-thai-bge-m3"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
if index_name not in existing_indexes:
    pc.create_index(index_name,dimension = 1024, metric = 'dotproduct', spec = spec)
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print(f"Create vector database name {index_name} successfull !")
else:
    print(f"Aready got vector database name {index_name} !!!")

In [ ]:
index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset(
    "Tun555/us-places-thai",
    split = "train"
)
data = dataset.to_pandas()
data["DESTINATION"] = data["DESTINATION_TH"]
data["NAME"] = data["NAME_TH"]
data["DESCRIPTION"] = data["DESCRIPTION_TH"]
data["REVIEWS"] = data["REVIEWS_TH"]
data = data[['RATING', 'PICTURE','URL', 'DESTINATION', 'NAME', 'DESCRIPTION', 'REVIEWS']]
data

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/135 [00:00<?, ? examples/s]

,RATING,PICTURE,URL,DESTINATION,NAME,DESCRIPTION,REVIEWS
0,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,พิพิธภัณฑ์ศิลปะบลานตัน,"{'categories': array(['แกลเลอรีศิลปะ', 'จุดน่า...","[""พิพิธภัณฑ์ศิลปะที่น่าทึ่ง เราได้ไปที่พิพิธภั..."
1,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,โรงละครโลกเดียว,"{'categories': array(['โรงละคร'], dtype=object...","[""สถานที่จัดคอนเสิร์ตที่ดีที่สุดในเท็กซัส นี่ค..."
2,4.0,None,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,ออสตินอะควาเรียม,"{'categories': array(['ตู้ปลา'], dtype=object)...","[""การเยี่ยมชมภายในบ้านที่ดีเหมาะกับครอบครัวขนา..."
3,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,"""Myo"" can be translated to Thai as ""มิโย"" if i...","{'categories': array(['สปา'], dtype=object), '...","[""ไม่มีการนวดที่แย่เลย ฉันได้ไปเยือนสตูดิโอนวด..."
4,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ออสติน,บาร์ตัน สปริงส์ พูล,"{'categories': array(['แหล่งน้ำ', 'น้ำพุร้อนแล...","[""คุณเคยเห็นซาลามานเดอร์ในสระน้ำไหม?, นี่ไม่ใช..."
...,...,...,...,...,...,...,...
130,4.5,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,สวนก๊าซ เวิร์กส์,"{'categories': array(['สวนสาธารณะ'], dtype=obj...","[""คุณต้องไปชมพระอาทิตย์ตกที่อุทยานแก๊สเวิร์ค! ..."
131,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,วงจรรีไซเคิล,"{'หมวดหมู่': array(['การเช่าอุปกรณ์'], dtype=o...","[""ประสบการณ์การเช่าจักรยานที่ยอดเยี่ยม จักรยาน..."
132,4.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,สวนประติมากรรมโอลิมปิก,"{'หมวดหมู่': array(['สวนสาธารณะ'], dtype=objec...","[""เดินง่ายพร้อมวิวสวยงาม นี่เป็นกิจกรรมที่ง่าย..."
133,5.0,https://dynamic-media-cdn.tripadvisor.com/medi...,https://www.tripadvisor.com/Attraction_Review-...,ซีแอตเทิล,Seattle Escape Games ترجم إلى Thai: เกมหนีในซี...,"{'categories': array(['เกมหลบหนี'], dtype=obje...","[""ยอดเยี่ยมสำหรับกิจกรรมสร้างทีมงาน เราจองเกมห..."


In [ ]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import math
import re, unicodedata, time

# data = dataset.to_pandas()
batch_size = 5

def is_missing(x):
    return x is None or (isinstance(x, float) and math.isnan(x))

def convert_to_native_types(obj):
    if obj is None:
        return ""
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, np.integer):
        return int(obj)
    if isinstance(obj, np.floating):
        return "" if np.isnan(obj) else float(obj)
    if isinstance(obj, (list, tuple)):
        return [convert_to_native_types(v) for v in obj]
    if isinstance(obj, dict):
        return {k: convert_to_native_types(v) for k, v in obj.items()}
    return obj

def create_safe_id(destination, name, idx):
    destination = unicodedata.normalize('NFKD', str(destination or "")).encode('ASCII','ignore').decode('ASCII')
    name        = unicodedata.normalize('NFKD', str(name or "")).encode('ASCII','ignore').decode('ASCII')
    destination = re.sub(r'[^a-zA-Z0-9]+','_', destination).strip('_')
    name        = re.sub(r'[^a-zA-Z0-9]+','_', name).strip('_')
    return f"{destination}_{name}_{idx}"

# Store valid records
valid_ids = []
valid_embeds = []
valid_metadata = []
skipped = 0

for i in tqdm(range(0, len(data), batch_size)):
    batch = data.iloc[i:min(len(data), i+batch_size)]
    for idx, row in batch.iterrows():
        try:
            # ✅ Handle both dict or str DESCRIPTION
            desc_obj = row.get('DESCRIPTION', "")
            if isinstance(desc_obj, dict):
                desc_text = desc_obj.get('description', "")
            elif isinstance(desc_obj, str):
                desc_text = desc_obj
            else:
                desc_text = ""

            if not desc_text.strip():
                desc_text = "No description available."

            name = row.get('NAME', "") or ""
            dest = row.get('DESTINATION', "") or ""
            rating = row.get('RATING', 0.0)
            reviews = row.get('REVIEWS', [])
            url = row.get('URL', "") or ""

            # Some datasets have Thai list reviews, so force to list
            if not isinstance(reviews, list):
                reviews = [str(reviews)]

            # ✅ Text for embedding
            text_to_embed = f"ชื่อสถานที่: {name}\nสถานที่ตั้ง: {dest}\nรายละเอียด: {desc_text}\nรีวิว: {' '.join(reviews)}"

            embed = embed_model.embed_documents([text_to_embed])[0]
            record_id = create_safe_id(dest, name, idx)

            metadata_entry = convert_to_native_types({
                'text': desc_text,
                'name': name,
                'location': dest,
                'rating': 0.0 if is_missing(rating) else float(rating),
                'reviews': reviews,
                'url': url
            })

            valid_ids.append(record_id)
            valid_embeds.append(embed)
            valid_metadata.append(metadata_entry)

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            skipped += 1

    # ✅ Upsert per batch
    if valid_ids:
        sub_batch_size = 50
        for k in range(0, len(valid_ids), sub_batch_size):
            k_end = min(len(valid_ids), k+sub_batch_size)
            try:
                payload = list(zip(
                    valid_ids[k:k_end],
                    valid_embeds[k:k_end],
                    valid_metadata[k:k_end]
                ))
                index.upsert(vectors=payload)
                print(f"✅ Successfully upserted {k_end - k} records")
            except Exception as e:
                print(f"⚠️ Error upserting batch {k}-{k_end}: {e}")
                if k < len(valid_ids):
                    print(f"First ID in batch: {valid_ids[k]}")
        valid_ids.clear()
        valid_embeds.clear()
        valid_metadata.clear()

print(f"✅ Done! Skipped {skipped} rows due to errors.")


  0%|          | 0/27 [00:00<?, ?it/s]

✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Successfully upserted 5 records
✅ Done! Skipped 0 rows due to errors.


<hr>

In [ ]:
!pip install -U langchain-pinecone

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Initialize Pinecone client
pc = Pinecone(api_key=pinecone_api)

# Connect to index (new API uses .index not .Index)
index = pc.Index("us-place-thai-bge-m3")   # still works in SDK
# or equivalently
# index = pc.index("us-places-ragv99")

# Create vector store wrapper
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embed_model,    # pass full embedding object, not .embed_query
    text_key="text"
)


In [ ]:
query = "อยากเที่ยว พิพิธภัณฑ์"
v = vectorstore.similarity_search(query, k = 3)
for i in v:
    print(i,end='\n')

page_content='{'categories': array(['พิพิธภัณฑ์ทางทหาร', 'พิพิธภัณฑ์เฉพาะทาง'], dtype=object), 'description': "พิพิธภัณฑ์ Wings Over Miami Air Museum เป็นการสดุดีให้แก่ทหารผ่านศึกและนักบินที่เป็นผู้บุกเบิกการบินพลเรือนและการบินทางทหาร ด้วยเป้าหมายนี้ พิพิธภัณฑ์จึงมีการจัดแสดงและทำการบินเครื่องบินทางทหารและเครื่องบินคลาสสิก เพื่อแบ่งปันประวัติศาสตร์อันยาวนานของการบิน นอกจากนี้ยังให้ความรู้เกี่ยวกับมรดกการบินและนำเสนอภาพรวมของตำนานนักบินที่เคยบินเครื่องบินเหล่านั้น นี่คือพิพิธภัณฑ์เล็ก ๆ ที่มีเสน่ห์ ซึ่งเปิดโอกาสให้ผู้เข้าชมได้ใกล้ชิดกับเครื่องบินคลาสสิกและเครื่องบินจากสงคราม มีกรณีจัดแสดงมากมายที่เต็มไปด้วยแบบจำลองและของที่ระลึกอื่น ๆ บ่อยครั้งที่ผู้เข้าชมจะได้เห็นกระบวนการฟื้นฟูเครื่องบิน และในวันหยุดสุดสัปดาห์ ผู้คนอาจมีโอกาสได้พูดคุยกับนักบินของเครื่องบินหากพวกเขาอยู่ในโรงเก็บเครื่องบิน แม้ว่าผู้เข้าชมจะไม่สามารถปีนขึ้นไปบนเครื่องบินได้ แต่กล้องถ่ายภาพได้รับการต้อนรับอย่างดี และทีมงานก็เป็นมิตรอย่างมาก."}' metadata={'location': 'ไมอามี', 'name': 'พิพิธภัณฑ์การบินวิงส์โอเวอร์ไมอามี', 

In [ ]:
query = "อยากเที่ยว พิพิธภัณฑ์"
results = vectorstore.similarity_search_with_score(query, k=3)

for doc, score in results:
    print(f"Score: {score:.4f}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print()


Score: 0.5767
Content: {'categories': array(['พิพิธภัณฑ์ทางทหาร', 'พิพิธภัณฑ์เฉพาะทาง'], dtype=object), 'description': "พิพิธภัณฑ์ Wings Over Miami Air Museum เป็นการสดุดีให้แก่ทหารผ่านศึกและนักบินที่เป็นผู้บุกเบิกการบินพลเรือนและการบินทางทหาร ด้วยเป้าหมายนี้ พิพิธภัณฑ์จึงมีการจัดแสดงและทำการบินเครื่องบินทางทหารและเครื่องบินคลาสสิก เพื่อแบ่งปันประวัติศาสตร์อันยาวนานของการบิน นอกจากนี้ยังให้ความรู้เกี่ยวกับมรดกการบินและนำเสนอภาพรวมของตำนานนักบินที่เคยบินเครื่องบินเหล่านั้น นี่คือพิพิธภัณฑ์เล็ก ๆ ที่มีเสน่ห์ ซึ่งเปิดโอกาสให้ผู้เข้าชมได้ใกล้ชิดกับเครื่องบินคลาสสิกและเครื่องบินจากสงคราม มีกรณีจัดแสดงมากมายที่เต็มไปด้วยแบบจำลองและของที่ระลึกอื่น ๆ บ่อยครั้งที่ผู้เข้าชมจะได้เห็นกระบวนการฟื้นฟูเครื่องบิน และในวันหยุดสุดสัปดาห์ ผู้คนอาจมีโอกาสได้พูดคุยกับนักบินของเครื่องบินหากพวกเขาอยู่ในโรงเก็บเครื่องบิน แม้ว่าผู้เข้าชมจะไม่สามารถปีนขึ้นไปบนเครื่องบินได้ แต่กล้องถ่ายภาพได้รับการต้อนรับอย่างดี และทีมงานก็เป็นมิตรอย่างมาก."}
Metadata: {'location': 'ไมอามี', 'name': 'พิพิธภัณฑ์การบินวิงส์โอเวอร์

In [ ]:
query = "อยากเที่ยวได้ รายละเอียดเกี่ยวกับการท่องเที่ยวแนวธรรมชาติ"
v = vectorstore.similarity_search(query, k = 3)
for i in v:
    print(i,end='\n')

page_content='{'categories': array(['เส้นทางจักรยาน', 'เส้นทางเดินป่า'], dtype=object), 'description': "เส้นทางจักรยานริมทะเลสาบชิคาโกมีความยาวประมาณ 22 ไมล์ โดยทอดยาวจากชายแดนอินเดียนาไปจนถึงเอแวนสตัน เส้นทางนี้ผ่านสวนมากมายรวมถึงสวนแจ็คสัน (ซึ่งมีมหาวิทยาลัยชิคาโก) สวนแกรนต์ (ซึ่งมีน้ำพุบัคกิงแฮม/พิพิธภัณฑ์แคมปัส) มิลเลนเนียมพาร์ค และลินคอล์นพาร์คที่มีสวนสัตว์ฟรี มีชายหาดและบริการให้เช่าจักรยานที่ Bobby's Bike Hike - 540 N. Lake Shore Drive, Chicago IL 60611 (มีแผนที่ฟรีและหมวกกันน็อก)." }' metadata={'location': 'ชิคาโก', 'name': 'Lakefront Trail สามารถแปลเป็นไทยได้ว่า "เส้นทางเลียบทะเลสาบ"', 'rating': 4.5, 'reviews': ['["เส้นทางที่สวยงาม! , การเช่าจักรยานจากบริการแชร์จักรยานของเมืองและปั่นจักรยานเกือบตลอดความยาวของเส้นทางนี้เป็นไฮไลท์ของการเดินทางของเราในที่นี่ เส้นทางจักรยานแน่นจริงในช่วงเวลาที่มีการจราจรหนาแน่น แต่ก็รู้สึกกระปรี้กระเปร่าที่ได้เห็นคนท้องถิ่นใช้เส้นทางจักรยานในลักษณะที่มีประสิทธิภาพ (บอสตันโปรดทราบ) วิวยอดเยี่ยมมาก , เดินทางมาเป็นคู่", "การท่องเที่ยวที่สนุกสนานในวัน

In [ ]:
i.page_content.split("array([")[1].split("],")[0]

"'พิพิธภัณฑ์เฉพาะทาง'"

In [ ]:
queries = [
    "อยากไปเที่ยวพิพิธภัณฑ์ในไมอามี่",
    "สถานที่เกี่ยวกับศิลปะและวัฒนธรรม" ,
    "อยากดูภาพจิตรกรรมฝาผนังหรือกราฟฟิตี้",
    "มีที่ไหนที่จัดแสดงผลงานศิลปะบ้าง",
    "อยากได้สถานที่ท่องเที่ยวที่เป็นมรดกทางวัฒนธรรม",
    "อยากเที่ยวกับครอบครัว",
    "สถานที่ที่เหมาะกับการพักผ่อน",
    "อยากไปถ่ายรูปสวยๆ",
    "สถานที่มีกิจกรรมสนุกๆ ให้ทำ",
    "อยากไปที่เหมาะกับเด็กๆ",
    "สถานที่ที่มีทั้งพิพิธภัณฑ์และสวนสวย",
    "อยากไปที่ที่มีสัตว์และสามารถเรียนรู้ได้ด้วย",
    "อยากเที่ยวแบบไม่ใช่ในตัวเมือง",
    "อยากได้ที่เที่ยวที่มีทั้งศิลปะและธรรมชาติ",
    "สถานที่ที่สามารถเรียนรู้ประวัติศาสตร์ของไมอามี่"
]
for q in queries:
    print(q)
    v = vectorstore.similarity_search_with_score(q,k=5)
    for i,score in v:
        page = str(i.page_content)
        if "array([" in page:
            try:
                cat_part = page.split("array([")[1].split("],")[0]
            except Exception:
                cat_part = "-"
        else:
            cat_part = "-"
        # print(i)
        print(f"- Place : {i.metadata.get('name','?')}")
        print(f"    - Score    : {score:.5f}")
        print(f"    - Category : {cat_part}")
        print(f"    - Position : {i.metadata.get('location','?')}")
        print(f"    - url      : {i.metadata.get('url','?')}")

    print()


อยากไปเที่ยวพิพิธภัณฑ์ในไมอามี่
- Place : พิพิธภัณฑ์เด็กไมอามี่
    - Score    : 0.70899
    - Category : "พิพิธภัณฑ์เด็ก"
    - Position : ไมอามี่
    - url      : https://www.tripadvisor.com/Attraction_Review-g34438-d547153-Reviews-Miami_Children_s_Museum-Miami_Florida.html
- Place : พิพิธภัณฑ์การบินวิงส์โอเวอร์ไมอามี
    - Score    : 0.66959
    - Category : 'พิพิธภัณฑ์ทางทหาร', 'พิพิธภัณฑ์เฉพาะทาง'
    - Position : ไมอามี
    - url      : https://www.tripadvisor.com/Attraction_Review-g34438-d263451-Reviews-Wings_Over_Miami_Air_Museum-Miami_Florida.html
- Place : พิพิธภัณฑ์รถไฟโกลด์โคสต์
    - Score    : 0.62189
    - Category : 'พิพิธภัณฑ์เฉพาะทาง'
    - Position : ไมอามี
    - url      : https://www.tripadvisor.com/Attraction_Review-g34438-d562881-Reviews-The_Gold_Coast_Railroad_Museum-Miami_Florida.html
- Place : สถานที่ประวัติศาสตร์แบร์นาเคิล
    - Score    : 0.61699
    - Category : 'สถานที่ประวัติศาสตร์'
    - Position : ไมอามี่
    - url      : https://www.tripadvisor.com/Att